In [1]:
import os

In [2]:
def create_project(directory):
    if not os.path.exists(directory):
        print("Creating project", directory)
        os.makedirs(directory)

In [4]:
def write_file(file_name,content):
    with open (file_name, 'w') as f:
        f.write(content)

In [5]:
def create_file(project_name,base_url):
    queue = project_name + '_queue.txt'
    crawled = project_name + '_crawled.txt'
    if not os.path.isfile(queue):
        write_file(queue, base_url)
    if not os.path.isfile(crawled):
        write_file(crawled,'')
        

In [9]:
def append_to_file(path,data):
    with open (path,'a') as f:
        f.write(data + '\n')
def delete_file_content(path):
    with open (path,'w') as f:
        pass

In [10]:
from urllib import parse
from html.parser import HTMLParser

In [38]:
class LinkFinder(HTMLParser):
    def __init__(self,base_url,page_url):
        super().__init__()
        self.base_url = base_url
        self.page_url = page_url
        self.links = set()
    def handle_starttag(self,tag,attrs):
        if tag == 'a':
            for (attribute,value) in attrs:
                if attribute == 'href':
                    url = parse.urljoin(self.base_url,value)
                    self.links.add(url)
    def page_links(self):
        return self.links
    def error(self,message):
        pass

In [12]:
def file_to_set(file_name):
    result = set()
    with open (file_name, 'rt') as f:
        for line in f:
            result.add(line.replace('\n',''))
        return result
def set_to_file(links,file):
    delete_file_content(file)
    for link in sorted(links):
        append_to_file(file,link)

In [30]:
from urllib.parse import urlparse
def get_sub_domain_name(url):
    try:
        return urlparse(url).netloc
    except:
        return ""
def get_domain_name(url):
    try:
        results = get_sub_domain_name(url).split('.')
        return results[-2] + '.' + results[-1]
    except:
        return ""

In [40]:
from urllib.request import urlopen
class Spider():
    project_name = ''
    base_url = ''
    domain_name = ''
    queue_file = ''
    crawled_file = ''
    queue = set()
    crawled = set()
    def __init__ (self,project_name, base_url,domain_name):
        Spider.project_name = project_name
        Spider.base_url = base_url
        Spider.domain_name = domain_name
        Spider.queue_file = Spider.project_name + 'queue.txt'
        Spider.crawled_file = Spider.project_name + 'crawled.txt'
        self.boot()
        self.crawl_page('First spider',Spider.base_url)
    @staticmethod
    def boot():
        create_project(Spider.project_name)
        create_file(Spider.project_name,Spider.base_url)
        Spider.queue = file_to_set(Spider.queue_file)
        Spider.crawled = file_to_set(Spider.crawled_file)
    @staticmethod
    def crawl_page(thread_name,page_url):
        if page_url not in Spider.crawled:
            print (thread_name + 'not crawling' + page_url)
            print ( 'Queue' + str(len(Spider.queue)) + ' | crawled '+ str(len(Spider.crawled)))
            Spider.add_link_to_queue(Spider.gather_link(page_url))
            Spider.queue.remove(page_url)
            Spider.crawled.add(page_url)
            Spider.update_files()
    @staticmethod
    def gather_links(page_url):
        html_string = ''
        try:
            response = urlopen(page_url)
            if response.getheader('Content-Type') == 'text/html':
                html_bytes = response.read()
                html_string = html_bytes.decode('utf-8')
            finder = LinkFinder(Spider.base_url,page_url)
            finder.feed(html_string)
        except:
            print ('Error: can not crawl page')
            return set()
        return finder.page_links()

In [42]:
project_name = 'thenewboston'
homepage = 'https://thenewboston.com/'
domain_name = get_domain_name(homepage)
queue_file = project_name + 'queue.txt'
crawled_file = project_name + 'crawled.txt'
number_of_threads = 8
queue = Queue()
Spider(project_name,homepage,domain_name)

Creating project thenewboston


FileNotFoundError: [Errno 2] No such file or directory: 'thenewbostonqueue.txt'

In [41]:
from queue import Queue

In [44]:
project_name = 'thenewboston'
homepage = 'https://thenewboston.com/'
domain_name = get_domain_name(homepage)
queue_file = project_name + 'queue.txt'

In [45]:
queue_file

'thenewbostonqueue.txt'